<a href="https://colab.research.google.com/github/shinmiura/diveintocode-ml/blob/master/Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Sprintの目的**


・フレームワークのコードを読めるようにする

・フレームワークを習得し続けられるようになる

・理論を知っている範囲をフレームワークで動かす

**どのように学ぶか**

前半はTensorFlowのExampleを動かします。後半ではKerasのコードを書いていきます。

In [15]:
# 必要なライブラリのインポート
import tensorflow as tf
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from keras import backend as K
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.optimizers import Adam
from sklearn.datasets import load_iris
from sklearn.preprocessing import OneHotEncoder
from keras.datasets import mnist

mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False

ImportError: ignored

# 2.公式Example

深層学習フレームワークには公式にさまざまなモデルのExampleコードが公開されています。



**【問題1】公式チュートリアルモデルを分担して実行**

TensorFLowの公式チュートリアルモデルを分担して実行してください。


以下の中から1人ひとつ選び実行し、その結果を簡単に発表してください。


models/tutorials at master · tensorflow/models



In [2]:
# MNIST読み込み
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# 正規化
x_train, x_test = x_train / 255.0, x_test / 255.0

11501568/11490434 [==============================] - 0s 0us/step


In [3]:
# モデル構造の定義
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

In [4]:
# 予測値を出してみる（1データだけ使用）
predictions = model(x_train[:1]).numpy()
predictions

array([[-0.3095453 ,  0.24228613,  0.54759437,  0.6319077 , -0.02013959,
        -0.08315508,  0.5115673 , -0.7505156 ,  0.37092865,  0.28147405]],
      dtype=float32)

In [5]:
# softmax関数に通す
tf.nn.softmax(predictions).numpy()

array([[0.0590203 , 0.10248463, 0.13907607, 0.15131055, 0.07882948,
        0.07401528, 0.13415475, 0.03797436, 0.11655407, 0.10658051]],
      dtype=float32)

In [6]:
# 損失関数定義
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [7]:
# 損失算出
loss_fn(y_train[:1], predictions).numpy()

2.6034837

In [8]:
# コンパイル（計算や予測を出来る状態にする）
model.compile(
    optimizer='adam',
    loss=loss_fn,
    metrics=['accuracy']
)

In [9]:
# 学習実行
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.2954 - accuracy: 0.9144
Epoch 2/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.1467 - accuracy: 0.9566
Epoch 3/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.1097 - accuracy: 0.9669
Epoch 4/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0886 - accuracy: 0.9727
Epoch 5/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0773 - accuracy: 0.9755


In [10]:
# 評価(損失とACC)
model.evaluate(x_test,  y_test, verbose=3)

[0.07364367693662643, 0.9769999980926514]

In [11]:
# モデルが直接確率を返してくれるようにする
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [12]:
# 試しに予測
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[4.1184434e-09, 1.4409312e-08, 4.5148860e-07, 6.0682629e-05,
        2.7483477e-11, 7.9381628e-08, 1.6549418e-14, 9.9993527e-01,
        1.3972226e-07, 3.3340805e-06],
       [2.0966846e-08, 3.4912582e-06, 9.9996102e-01, 2.9429148e-05,
        5.5252144e-17, 5.9945373e-06, 2.0996407e-09, 1.9449887e-14,
        1.1818390e-07, 1.0981550e-14],
       [8.6785320e-08, 9.9924135e-01, 6.9798072e-05, 1.0899768e-05,
        5.3641583e-05, 3.5467365e-05, 1.3372746e-05, 5.0384179e-04,
        6.8875845e-05, 2.7016511e-06],
       [9.9942744e-01, 2.0519492e-10, 2.1406176e-04, 1.5533594e-07,
        1.7750862e-06, 3.3589859e-07, 1.7191243e-06, 8.0020955e-06,
        1.7054231e-07, 3.4645759e-04],
       [1.4293760e-05, 2.5113114e-09, 1.6740672e-05, 2.8387521e-07,
        9.7781634e-01, 1.9487486e-06, 1.3723566e-05, 2.4483472e-04,
        1.9272456e-07, 2.1891553e-02]], dtype=float32)>

**【問題2】（アドバンス課題）さまざまな手法を実行**

TensorFLowやGoogle AI ResearchのGitHubリポジトリには、定番のモデルから最新のモデルまで多様なコードが公開されています。これらから興味あるものを選び実行してください。


なお、これらのコードは初学者向けではないため、巨大なデータセットのダウンロードが必要な場合など、実行が簡単ではないこともあります。そういった場合は、コードリーディングを行ってください。


models/research at master · tensorflow/models


google-research/google-research: Google AI Research


更新日が古いものはPythonやTensorFlowのバージョンが古く、扱いずらい場合があります。新しいものから見ることを推奨します。

# 3.異なるフレームワークへの書き換え

「Sprint13 TensorFlow」で作成した4種類のデータセットを扱うTensorFLowのコードを異なるフレームワークに変更していきます。


・Iris（Iris-versicolorとIris-virginicaのみの2値分類）

・Iris（3種類すべての目的変数を使用して多値分類）

・House Prices

・MNIST

**Kerasへの書き換え**

KerasはTensorFLowに含まれるtf.kerasモジュールを使用してください。


KerasにはSequentialモデルかFunctional APIかなど書き方に種類がありますが、これは指定しません。



**【問題3】Iris（2値分類）をKerasで学習**

TensorFlowによるIrisデータセットに対する2値分類をKerasに書き換えてください。

In [16]:
# データ準備
# ライブラリの読み込み
from sklearn.datasets import load_iris
# 読み込み
iris_dataset = load_iris()

# 整形
iris_dataframe = pd.DataFrame(data=iris_dataset.data, columns=iris_dataset.feature_names)
iris_datalabel = pd.DataFrame(data=iris_dataset.target,columns=['Species'])
df = pd.concat([iris_dataframe,iris_datalabel],axis=1)

In [17]:
# 2値分類のため絞り込み
df2 = df[(df["Species"] == 0)|(df["Species"] == 1)]

# 説明変数と目的変数に分割
y = df2["Species"]
X = df2.loc[:, ["sepal length (cm)", "sepal width (cm)", "petal length (cm)", "petal width (cm)"]]
y = np.array(y)
X = np.array(X)
y = y.astype(np.int)[:, np.newaxis]

# 訓練データ/テストデータ/評価データに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

# 正規化
from sklearn.preprocessing import MinMaxScaler
mmsc = MinMaxScaler()
X_train = mmsc.fit_transform(X_train)
X_test = mmsc.transform(X_test)
X_val = mmsc.transform(X_val)

In [18]:
# モデル定義
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(50, activation = tf.nn.relu, input_shape=(4,)))
model.add(tf.keras.layers.Dense(100, activation = tf.nn.relu))
model.add(tf.keras.layers.Dense(1, activation = tf.nn.sigmoid))

In [19]:
# モデル構造の確認
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 50)                250       
_________________________________________________________________
dense_3 (Dense)              (None, 100)               5100      
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 101       
Total params: 5,451
Trainable params: 5,451
Non-trainable params: 0
_________________________________________________________________


In [21]:
# 損失が3回改善しなかったら早期学習終了
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

In [22]:
# モデルのコンパイル
model.compile(
    loss='binary_crossentropy',
    optimizer=tf.optimizers.Adam(learning_rate=0.01),
    metrics=['accuracy']
)

In [23]:
# 学習実行
model.fit(
    X_train,
    y_train,
    batch_size=10,
    epochs=100,
    verbose=1,
    callbacks=[callback],
    validation_data=(X_val, y_val)
)

Epoch 1/100
7/7 [==============================] - 1s 24ms/step - loss: 0.4562 - accuracy: 0.7656 - val_loss: 0.1538 - val_accuracy: 1.0000
Epoch 2/100
7/7 [==============================] - 0s 5ms/step - loss: 0.0680 - accuracy: 1.0000 - val_loss: 0.0062 - val_accuracy: 1.0000
Epoch 3/100
7/7 [==============================] - 0s 5ms/step - loss: 0.0039 - accuracy: 1.0000 - val_loss: 3.7891e-04 - val_accuracy: 1.0000
Epoch 4/100
7/7 [==============================] - 0s 5ms/step - loss: 4.2691e-04 - accuracy: 1.0000 - val_loss: 9.3879e-05 - val_accuracy: 1.0000
Epoch 5/100
7/7 [==============================] - 0s 5ms/step - loss: 1.2577e-04 - accuracy: 1.0000 - val_loss: 2.7345e-05 - val_accuracy: 1.0000
Epoch 6/100
7/7 [==============================] - 0s 5ms/step - loss: 5.7253e-05 - accuracy: 1.0000 - val_loss: 1.0457e-05 - val_accuracy: 1.0000
Epoch 7/100
7/7 [==============================] - 0s 5ms/step - loss: 3.1787e-05 - accuracy: 1.0000 - val_loss: 5.8635e-06 - val_accurac

In [24]:
# 2値分類のうちどちらに分類されるか
y_pred_proba = model.predict(X_test)[:, 0]
y_pred = np.where(y_pred_proba >0.5, 1, 0)
print("y_pred_proba", y_pred_proba)
print("y_pred", y_pred)
print('y_test', y_test.ravel())

y_pred_proba [4.1226301e-07 1.0000000e+00 6.7178132e-07 1.0000000e+00 1.0000000e+00
 9.9999464e-01 8.9222176e-09 1.0000000e+00 1.0000000e+00 9.9999905e-01
 1.0000000e+00 9.9999988e-01 1.0000000e+00 2.7369952e-06 1.8225766e-07
 1.4536547e-06 8.0100556e-08 4.1939347e-07 1.2356399e-09 5.9336198e-06]
y_pred [0 1 0 1 1 1 0 1 1 1 1 1 1 0 0 0 0 0 0 0]
y_test [0 1 0 1 1 1 0 1 1 1 1 1 1 0 0 0 0 0 0 0]


In [25]:
# 損失とACC
score = model.evaluate(X_test, y_test, verbose=0)
print('test loss:', score[0])
print('test accuracy:', score[1])

test loss: 9.155772886515479e-07
test accuracy: 1.0


**【問題4】Iris（多値分類）をKerasで学習**

TensorFlowによるIrisデータセットに対する3値分類をKerasに書き換えてください。

In [27]:
from sklearn.preprocessing import OneHotEncoder
# データ準備
# 説明変数と目的変数に分割
y = df["Species"]
X = df.loc[:, ["sepal length (cm)", "sepal width (cm)", "petal length (cm)", "petal width (cm)"]]
y = np.array(y)
X = np.array(X)
y = y.astype(np.int)[:, np.newaxis]

# 訓練データ/テストデータ/評価データに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

# onehotベクトル化
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
y_train_one_hot = enc.fit_transform(y_train)
y_val_one_hot = enc.transform(y_val)
y_test_one_hot = enc.transform(y_test)

# 正規化
from sklearn.preprocessing import MinMaxScaler
mmsc = MinMaxScaler()
X_train = mmsc.fit_transform(X_train)
X_test = mmsc.transform(X_test)
X_val = mmsc.transform(X_val)

In [28]:
# モデル定義
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(50, activation = tf.nn.relu, input_shape=(4,)))
model.add(tf.keras.layers.Dense(100, activation = tf.nn.relu))
model.add(tf.keras.layers.Dense(3, activation = tf.nn.softmax))

In [ ]:
# モデル構造の確認
model.summary()

In [30]:
# 損失が3回改善しなかったら早期学習終了
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

In [31]:
# モデルのコンパイル
model.compile(
    loss='categorical_crossentropy',
    optimizer=tf.optimizers.Adam(learning_rate=0.01),
    metrics=['accuracy']
)

In [32]:
# 学習実行
model.fit(
    X_train,
    y_train_one_hot,
    batch_size=10,
    epochs=100,
    verbose=1,
    callbacks=[callback],
    validation_data=(X_val, y_val_one_hot)
)

Epoch 1/100
10/10 [==============================] - 1s 31ms/step - loss: 0.9991 - accuracy: 0.4896 - val_loss: 0.7049 - val_accuracy: 0.7083
Epoch 2/100
10/10 [==============================] - 0s 4ms/step - loss: 0.5489 - accuracy: 0.7083 - val_loss: 0.4443 - val_accuracy: 0.8333
Epoch 3/100
10/10 [==============================] - 0s 4ms/step - loss: 0.4144 - accuracy: 0.7812 - val_loss: 0.4099 - val_accuracy: 0.7083
Epoch 4/100
10/10 [==============================] - 0s 4ms/step - loss: 0.3465 - accuracy: 0.8229 - val_loss: 0.2439 - val_accuracy: 0.9583
Epoch 5/100
10/10 [==============================] - 0s 5ms/step - loss: 0.2044 - accuracy: 0.9479 - val_loss: 0.1812 - val_accuracy: 0.9583
Epoch 6/100
10/10 [==============================] - 0s 4ms/step - loss: 0.1798 - accuracy: 0.9375 - val_loss: 0.2742 - val_accuracy: 0.9167
Epoch 7/100
10/10 [==============================] - 0s 4ms/step - loss: 0.1328 - accuracy: 0.9583 - val_loss: 0.2818 - val_accuracy: 0.8750
Epoch 8/100


In [33]:
# 学習実行
model.fit(
    X_train,
    y_train_one_hot,
    batch_size=10,
    epochs=100,
    verbose=1,
    callbacks=[callback],
    validation_data=(X_val, y_val_one_hot)
)

Epoch 1/100
10/10 [==============================] - 0s 7ms/step - loss: 0.0490 - accuracy: 0.9792 - val_loss: 0.1049 - val_accuracy: 1.0000
Epoch 2/100
10/10 [==============================] - 0s 4ms/step - loss: 0.0911 - accuracy: 0.9583 - val_loss: 0.1022 - val_accuracy: 1.0000
Epoch 3/100
10/10 [==============================] - 0s 4ms/step - loss: 0.0862 - accuracy: 0.9583 - val_loss: 0.8272 - val_accuracy: 0.7917
Epoch 4/100
10/10 [==============================] - 0s 4ms/step - loss: 0.2097 - accuracy: 0.9375 - val_loss: 0.1150 - val_accuracy: 0.9583


In [34]:
# 損失とACC
score = model.evaluate(X_test, y_test_one_hot, verbose=0)
print('Train loss:', score[0])
print('Train accuracy:', score[1])

Train loss: 0.06182991713285446
Train accuracy: 0.9666666388511658


**【問題5】House PricesをKerasで学習**

TensorFlowによるHouse Pricesデータセットに対する回帰をKerasに書き換えてください。

In [36]:
%pwd

'/content'

In [43]:
%ls

 シンプソン/
 13_クラスとインスタンス.ipynb
 15_クラス変数とメンバ変数.ipynb
 16_クラスの使い道.ipynb
 Chapter1.ipynb
 Chapter2.ipynb
 CIFAR10_transfer_learning.ipynb
'CNN_intro (1).ipynb'
 CNN_intro_answer.ipynb
 CNN_intro.ipynb
 data_handling_practice_answer.ipynb
 deep-learning-from-scratch-2-master/
 deep-learning-from-scratch-master/
 def関数練習.ipynb
'example_answer_week3_sinyo_joho (1).ipynb'
 example_answer_week3_sinyo_joho.ipynb
'example_answer_week4_ayame (1).ipynb'
 example_answer_week4_ayame.ipynb
 example_answer_week4_jutaku_price.ipynb
 example_answer_week4_shinyo_joho.ipynb
 H2O_for_preparation.ipynb
 H2O_walk_through.ipynb
 homework2_transfer_learning_answer.ipynb
 homework2_transfer_learning.ipynb
 クラスタリング.ipynb
 クラスとインスタンス.ipynb
 ニューラルネットワーク.ipynb
 ディープニューラルネットワーク.ipynb
'janome_practice_NLP (1).ipynb'
 janome_practice_NLP.ipynb
 Keras.ipynb
 Keras入門.ipynb
 Linear_regression.ipynb
'MEF_Sprint18_make_dataset（受講生用） (1).ipynb'
 MEF_Sprint18_make_dataset（受講生用）.ipynb
 modeling_process_walk_through.ipynb


In [42]:
%cd Colab Notebooks

/content/drive/My Drive/Colab Notebooks


In [44]:
# データの準備
dataset_path ="train.csv"
df = pd.read_csv(dataset_path)

y = df["SalePrice"]
X = df.loc[:, ["GrLivArea", "YearBuilt"]]
y = np.array(y)
X = np.array(X)
y = y.astype(np.int)[:, np.newaxis]
y = np.log(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

In [45]:
# モデルの定義
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(50, activation = tf.nn.relu, input_shape=(2,)))
model.add(tf.keras.layers.Dense(100, activation = tf.nn.relu))
model.add(tf.keras.layers.Dense(1, activation = tf.keras.activations.linear))

In [46]:
# モデル構造の確認
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 50)                150       
_________________________________________________________________
dense_9 (Dense)              (None, 100)               5100      
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 101       
Total params: 5,351
Trainable params: 5,351
Non-trainable params: 0
_________________________________________________________________


In [47]:
# 損失が3回改善しなかったら早期学習終了
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

In [48]:
# モデルのコンパイル
model.compile(
    loss='mse',
    optimizer='adam',
    metrics=['mse']
)

In [49]:
# 学習実行
model.fit(
    X_train,
    y_train,
    batch_size=10,
    epochs=100,
    verbose=1,
    callbacks=[callback],
    validation_data=(X_val, y_val)
)

Epoch 1/100
94/94 [==============================] - 1s 4ms/step - loss: 4122.5811 - mse: 4122.5811 - val_loss: 12.6900 - val_mse: 12.6900
Epoch 2/100
94/94 [==============================] - 0s 3ms/step - loss: 6.9242 - mse: 6.9242 - val_loss: 3.8037 - val_mse: 3.8037
Epoch 3/100
94/94 [==============================] - 0s 3ms/step - loss: 2.7022 - mse: 2.7022 - val_loss: 2.3352 - val_mse: 2.3352
Epoch 4/100
94/94 [==============================] - 0s 3ms/step - loss: 1.4058 - mse: 1.4058 - val_loss: 0.9299 - val_mse: 0.9299
Epoch 5/100
94/94 [==============================] - 0s 3ms/step - loss: 0.8406 - mse: 0.8406 - val_loss: 1.2936 - val_mse: 1.2936
Epoch 6/100
94/94 [==============================] - 0s 2ms/step - loss: 0.6240 - mse: 0.6240 - val_loss: 0.4142 - val_mse: 0.4142
Epoch 7/100
94/94 [==============================] - 0s 3ms/step - loss: 0.4960 - mse: 0.4960 - val_loss: 0.4375 - val_mse: 0.4375
Epoch 8/100
94/94 [==============================] - 0s 2ms/step - loss: 0.

In [50]:
# 予測値出力
y_pred_proba = model.predict(X_test)
y_pred = y_pred_proba
print("y_pred", y_pred.ravel()[:5])
print('y_test', y_test.ravel()[:5])

y_pred [12.23102  11.922037 11.775415 12.30884  11.850685]
y_test [12.20918779 11.79810441 11.60823564 12.16525065 11.38509209]


In [51]:
# 損失とACC
score = model.evaluate(X_test, y_test, verbose=0)
print('loss (test):', score[0])
print('Mean Squared Error (test):', score[1])

loss (test): 0.6131235361099243
Mean Squared Error (test): 0.6131235361099243


**【問題6】MNISTをKerasで学習**

TensorFlowによるMNISTデータセットによる画像の多値分類をKerasに書き換えてください。

In [52]:
# データ準備
# 読み込み
(X_train, y_train), (X_test, y_test) = mnist.load_data()

#　平滑化
X_train = X_train.reshape(-1, 784)
X_test = X_test.reshape(-1, 784)

# 正規化
X_train = X_train.astype(np.float)
X_test = X_test.astype(np.float)
X_train /= 255
X_test /= 255

# 変形
y_train = y_train.astype(np.int)[:, np.newaxis]
y_test = y_test.astype(np.int)[:, np.newaxis]

# one-hotベクトル化
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
y_train_one_hot = enc.fit_transform(y_train[:])
y_test_one_hot = enc.fit_transform(y_test[:])

# 分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train_one_hot, test_size=0.2)

In [53]:
# モデル定義
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(50, activation = tf.nn.relu, input_shape=(784,)))
model.add(tf.keras.layers.Dense(100, activation = tf.nn.relu))
model.add(tf.keras.layers.Dense(10, activation = tf.nn.softmax))

In [54]:
# 損失が3回改善しなかったら早期学習終了
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

In [55]:
# モデルのコンパイル
model.compile(
    loss='categorical_crossentropy',
    optimizer=tf.optimizers.Adam(learning_rate=0.01),
    metrics=['accuracy']
)

In [56]:
# 学習実行
model.fit(
    X_train[:1000],
    y_train[:1000],
    batch_size=10,
    epochs=100,
    verbose=1,
    callbacks=[callback],
    validation_data=(X_val, y_val)
)

Epoch 1/100
100/100 [==============================] - 3s 29ms/step - loss: 1.0017 - accuracy: 0.6520 - val_loss: 0.6028 - val_accuracy: 0.8195
Epoch 2/100
100/100 [==============================] - 3s 28ms/step - loss: 0.4510 - accuracy: 0.8550 - val_loss: 0.6299 - val_accuracy: 0.8253
Epoch 3/100
100/100 [==============================] - 3s 28ms/step - loss: 0.2988 - accuracy: 0.9050 - val_loss: 0.5779 - val_accuracy: 0.8404
Epoch 4/100
100/100 [==============================] - 2s 15ms/step - loss: 0.2677 - accuracy: 0.9080 - val_loss: 0.6925 - val_accuracy: 0.8172
Epoch 5/100
100/100 [==============================] - 2s 15ms/step - loss: 0.1603 - accuracy: 0.9570 - val_loss: 0.5259 - val_accuracy: 0.8756
Epoch 6/100
100/100 [==============================] - 2s 16ms/step - loss: 0.1384 - accuracy: 0.9590 - val_loss: 0.6216 - val_accuracy: 0.8655
Epoch 7/100
100/100 [==============================] - 3s 28ms/step - loss: 0.1238 - accuracy: 0.9660 - val_loss: 0.8068 - val_accuracy:

In [57]:
# 予測値出力
y_pred_proba = model.predict(X_test)
y_pred = np.argmax(y_pred_proba, axis=1)
print("y_pred", y_pred[:10])
print('y_test', y_test.ravel()[:10])

y_pred [7 2 1 0 4 1 4 9 6 9]
y_test [7 2 1 0 4 1 4 9 5 9]


In [58]:
# 損失とACC
score = model.evaluate(X_test, y_test_one_hot, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.6227262020111084
Test accuracy: 0.8758999705314636


**【問題7】（アドバンス課題）PyTorchへの書き換え**

4種類の問題をPyTorchに書き換えてください。


pytorch:tutorials



**【問題8】（アドバンス課題）フレームワークの比較**

それぞれのフレームワークにはどのような違いがあるかをまとめてください。


《視点例》


・計算速度

・コードの行数・可読性

・用意されている機能